# DAT500 - FinalProject, Flight Delay Prediction

Group Members:

    - Brage Solheim
    - Yohannes D. Kassaye

## Introduction

So far we have used map reduce jobs to find null values in our data, find mean and mode of each column, and fill null values with mean or mode.

## Initlization

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark'

In [2]:
from datetime import datetime, date
import pandas as pd
import numpy as np
from pyspark.sql import Row
from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [3]:
ORIGINAL_DATA_PATH = "hdfs:///data/airline_delay_data.csv"
CLEANED_DATA_PATH = "hdfs:///data/airline_delay_data_cleaned.csv"

TEST_DATA_ORIGINAL_PATH = "/data/2015.sample.csv"
TEST_DATA_CLEANED_PATH = "/data/2015.sample.cleaned.csv"

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pyspark
conf = pyspark.SparkConf().setAll([("spark.driver.maxResultSize", '4g'), ("spark.sql.execution.arrow.pyspark.enabled", 'true')])

In [6]:
spark = (SparkSession
            .builder
            .master("yarn") # for cluster
            # .master("local[*]") # for local testing
            .config(conf=conf)
            .appName("Flight-delay-prediction")
            .getOrCreate())

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-18 16:42:50,443 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-18 16:42:54,124 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:
print(sc.getConf().getAll())

[('spark.ui.proxyBase', '/proxy/application_1645016454320_0101'), ('spark.sql.warehouse.dir', 'file:/home/ubuntu/DAT500-FinalProject/spark/spark-warehouse'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Flight-delay-prediction'), ('spark.app.id', 'application_1645016454320_0101'), ('spark.driver.host', 'namenode'), ('spark.master', 'yarn'), ('spark.app.startTime', '1650300170317'), ('spark.executorEnv.PYTHONPATH', '/usr/local/spark/python/:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9.3-src.zip'), ('spark.rdd.compress', 'True'), ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES', 'http://namenode:8088/proxy/application_1645016454320_0101'), ('spark.serializer.objectStreamReset', '100'), ('spark.submit.pyFiles', ''), ('spark.yarn.isPython', 'true'), ('spark.driver.port', '41297'), ('spark.submit.deployMode', 'client'), ('spark.ui.filters', 'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'), ('spark.org.apache.hadoop.yarn.serve

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType

In [21]:
features = ["FL_DATE","OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_TIME","DEP_DELAY","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY","CANCELLED","DIVERTED","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME","DISTANCE","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
nullable=False
schema = StructType([
    StructField("FL_DATE", DateType(), nullable), StructField("OP_CARRIER", StringType(), nullable), StructField("OP_CARRIER_FL_NUM", DoubleType(), nullable),
    StructField("ORIGIN", StringType(), nullable), StructField("DEST", StringType(), nullable), StructField("CRS_DEP_TIME", DoubleType(), nullable),
    StructField("DEP_TIME", DoubleType(), nullable), StructField("DEP_DELAY", DoubleType(), nullable), StructField("TAXI_OUT", DoubleType(), nullable),
    StructField("WHEELS_OFF", DoubleType(), nullable), StructField("WHEELS_ON", DoubleType(), nullable), StructField("TAXI_IN", DoubleType(), nullable),
    StructField("CRS_ARR_TIME", DoubleType(), nullable), StructField("ARR_TIME", DoubleType(), nullable), StructField("ARR_DELAY", DoubleType(), nullable),
    StructField("CANCELLED", DoubleType(), nullable), StructField("DIVERTED", DoubleType(), nullable), StructField("CRS_ELAPSED_TIME", DoubleType(), nullable),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), nullable), StructField("AIR_TIME", DoubleType(), nullable), StructField("DISTANCE", DoubleType(), nullable),
    StructField("CARRIER_DELAY", DoubleType(), nullable), StructField("WEATHER_DELAY", DoubleType(), nullable), StructField("NAS_DELAY", DoubleType(), nullable),
    StructField("SECURITY_DELAY", DoubleType(), nullable), StructField("LATE_AIRCRAFT_DELAY", DoubleType(), nullable)])

In [22]:
df = spark.read.csv(CLEANED_DATA_PATH, header=True, schema=schema, inferSchema=False)
df.show(2)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+
|2016-09-25|        AA|           2622.0|   EGE| DFW|       800.0|   749.0|    -11.0|    11

In [23]:
df = df.repartition(24)

In [24]:
from pyspark.sql import functions as F

In [25]:
header_row = df.filter(F.col("FL_DATE").isNull())
header_row.to_pandas_on_spark()

2022-04-18 16:52:06,658 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-18 16:52:06,661 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-18 16:52:56,566 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,None,OP_CARRIER,NaN,ORIGIN,DEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see that there is one row with null values do to the header being in the middle of our data, and when we infer the schema, the header row is not the same type as the rest of the rows, and therefore is converted to null.

In [26]:
df = df.filter((F.col("FL_DATE").isNull() & F.col("OP_CARRIER").isNotNull() & F.col("ORIGIN").isNotNull() & F.col("DEST").isNotNull()) == False)

In [27]:
header_row = df.filter(F.col("FL_DATE").isNull())
print(header_row.count())

0


## Data Analysis

So far we have cleaned the data by removing and filling null values.
Now we will try to find the correlation between the our target, DEP_DELAY, and the other columns.

In [16]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [17]:
def GetCorrelationMatrix(df):
    inputcols = [col[0] for col in df.dtypes if col[1] == "double" or col[1] == "int"]
    vector_col = "features"

    assembler = VectorAssembler(inputCols=inputcols, outputCol=vector_col, handleInvalid="skip")
    df_vector = assembler.transform(df).select(vector_col)
    matrix = Correlation.corr(df_vector, "features")

    matrix_df = spark.createDataFrame(matrix.collect()[0]["pearson({})".format(vector_col)].toArray().tolist(), inputcols).to_pandas_on_spark()
    matrix_df["feature"] = inputcols

    return matrix_df
    

In [18]:
corr_matrix = GetCorrelationMatrix(df)

In [19]:
corr_matrix[["DEP_DELAY", "feature"]].sort_values(by="DEP_DELAY", ascending=False)

2022-04-18 16:48:44,819 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-18 16:48:45,239 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,DEP_DELAY,feature
3,1.000000,DEP_DELAY
10,0.947662,ARR_DELAY
17,0.524859,CARRIER_DELAY
21,0.420768,LATE_AIRCRAFT_DELAY
18,0.215824,WEATHER_DELAY
2,0.147498,DEP_TIME
5,0.137768,WHEELS_OFF
19,0.121006,NAS_DELAY
1,0.105527,CRS_DEP_TIME
8,0.090892,CRS_ARR_TIME


We see that there are no negative correlations between the target and the other columns. So we dont need to remove any columns.